# Metaclass

In [6]:
class MyType(type):
    def __new__(cls, clsname, bases, clsdict):
        if len(bases) > 1:
            print(bases)
            raise TypeError("No!!")
        return super().__new__(cls, clsname, bases, clsdict)

class DebugInfo(type):
    def __new__(cls, clsname, bases, clsdict):
        print(f"Debug Info {clsname}, {clsdict}")
        return super().__new__(cls, clsname, bases, clsdict)
        
    
class Base(metaclass=MyType):
    pass

# good
class A(Base):
    pass

# good
class B(Base):
    pass

# error!!
class C(A, B):
    pass

Debug Info Base, {'__module__': '__main__', '__qualname__': 'Base'}
Debug Info A, {'__module__': '__main__', '__qualname__': 'A'}
Debug Info B, {'__module__': '__main__', '__qualname__': 'B'}
Debug Info C, {'__module__': '__main__', '__qualname__': 'C'}


In [29]:
def debugattr(cls):
    orig_getattribute = cls.__getattribute__

    def __getattribute__(self, name):
        print(f'Get: {name}')
        return orig_getattribute(self, name)

    cls.__getattribute__ = __getattribute__

    return cls


class DebugInfo(type):
    def __new__(cls, clsname, bases, clsdict):
        clsobj = super().__new__(cls, clsname, bases, clsdict)
        clsobj = debugattr(clsobj)
        return clsobj

# Use Class Decorator
@debugattr
class A:
    def __init__(self, a_x):
        self.a_x = a_x

# Use metaclass
class B(metaclass=DebugInfo):
    def __init__(self, b_x):
        self.b_x = b_x


a = A(1)
a.a_x

b = B(1)
b.b_x

Get: a_x
Get: b_x


1

## memo
Decorater -> Functions  
Class Decorators  -> Classes  
Meta Classes -> Class Hieerarchiese  

# Data Structure

## とりあえず

In [41]:
class Structure:
    _fields = []

    def __init__(self, *args):
        for name, val in zip(self._fields, args):
            setattr(self, name, val)


class Stock(Structure):
    _fields = ['name', 'shares', 'price']
    #     def __init__(self, name, shares, price)
    #         self.name = name
    #         self.shares = shares
    #         self.price = price
    
st = Stock('Goog', 100, 490.1)
print(st.__dict__)

# Attrが足りなくても作れる
st2 = Stock('Goog', 100)
print(st2.__dict__)

# kwargとか使えない
st3 = Stock(name='Goog')
print(st3.__dict__)


{'name': 'Goog', 'shares': 100, 'price': 490.1}
{'name': 'Goog', 'shares': 100}


TypeError: __init__() got an unexpected keyword argument 'name'

## もう少し足す

Blog post fo Parameter, Signature http://blog.amedama.jp/entry/2016/10/31/225219  
Parameter https://docs.python.org/3/library/inspect.html#inspect.Parameter
Signature https://docs.python.org/3/library/inspect.html#inspect.Signature

In [60]:
from inspect import Parameter, Signature


def make_signature(names):
    return Signature(
        Parameter(
            name=name,
            kind=Parameter.POSITIONAL_OR_KEYWORD # https://docs.python.org/3/library/inspect.html#inspect.Parameter.kind
        )
        for name in names
    )


class Structure:
    __signature__ = make_signature([])

    def __init__(self, *args, **kwargs):
        bound = self.__signature__.bind(*args, **kwargs)
        for name, val in bound.arguments.items():
            setattr(self, name, val)

class Stock(Structure):
    __signature__ = make_signature(['name', 'shares', 'price'])

st = Stock('Goog', 100, 490.1)
print(st.__dict__)

st3 = Stock(name='Goog', shares=100, price=490)
print(st3.__dict__)

# argが足りないと壊れる
st4 = Stock(name='Goog', shares=100)


{'name': 'Goog', 'shares': 100, 'price': 490.1}
{'name': 'Goog', 'shares': 100, 'price': 490}


TypeError: missing a required argument: 'price'

# もっと直す

In [62]:
from inspect import Parameter, Signature

def make_signature(names):
    return Signature(
        Parameter(name, Parameter.POSITIONAL_OR_KEYWORD)
        for name in names)

class StructMeta(type):
    def __new__(cls, clsname, bases, clsdict):
        clsobj = super().__new__(cls, clsname, bases, clsdict)
        sig = make_signature(clsobj._fields)
        setattr(clsobj, '__signature__', sig)
        return clsobj

class Structure(metaclass=StructMeta):
    _fields = []
    def __init__(self, *args, **kwargs):
        bound = self.__signature__.bind(*args, **kwargs)
        for name, val in bound.arguments.items():
            setattr(self, name, val)

class Stock(Structure):
    _fields = ['name', 'shares', 'price']

class Point(Structure):
    _fields = ['x', 'y']

class Host(Structure):
    _fields = ['address', 'port']


st = Stock(name='AAA', shares=100, price='400')
st.__dict__

{'name': 'AAA', 'price': '400', 'shares': 100}

## Memo
Class Decorator => Goal is to tweak class that might be unrelated   
Metaclass -> trying to perform actions in combination with inheritance